## Homework 5

### Complex capacity expansion planning

In this homework assignment, we will explore the workings of a complex electricity system capacity expansion planning model that includes chronologically sequential operational decisions with time coupling constraints and transport flow constraints to represent power transmission limits between multiple geospatial regions.

The 'core' model and data we will use are provided in [Notebooks/extra_labs/Lab1.jl](https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/extra_labs/Lab1.jl), [Notebooks/extra_labs/Lab1_code.jl](https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/extra_labs/Lab1_code.jl) and the [complex_expansion_data](https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/tree/master/Notebooks/complex_expansion_data) folder. This core model includes economic dispatch decisions with ramp constraints for thermal generators and storage constraints (**but no thermal unit commitment constraints/decisions**). It uses a simplified capacitated transport flow constraint to represent inter-regional electricity transmission between three regions (rather than DC OPF constraints), with data representing the Electricity Reliability Corporation of Texas (ERCOT) region.

### Part 1: Time domain reduction methods

To reduce dimensionality and keep runtimes manageable for a tutorial assignment running on a desktop or laptop computer, we provide a set of *N* representative time periods of *Y* consecutive hourly periods (here either 24 hour days or 168 hour weeks) selected via a clustering method adapted from Mallapragada et al. (2018), "[Impact of model resolution on scenario outcomes for electricity sector system expansion](https://doi.org/10.1016/j.energy.2018.08.015)" Energy 163). 

This method creates clusters of representative periods by trying to minimize variation between time series for wind, solar, and demand in each of the within-cluster periods (days or weeks). It also always includes the period containing the peak demand hour, if this period was not already selected as a cluster centroid. 

To represent a full year of hourly operations, hours within each time period are weighted by a multiplier equal to the total number of periods (days or weeks) within the cluster represented by each representative period. In the objective function of the model, variable costs incurred in each hour are thus multiplied by this hourly weight to represent the cost of a full year of operation, as shown in the following two expressions, which make up the variable cost related part of the model objective function: 

In [ ]:
# @expression(Expansion_Model, eVariableCosts,
#      # Variable costs for generation, weighted by hourly sample weight
#     sum(sample_weight[t]*generators.Var_Cost[g]*vGEN[t,g] for t in T, g in G)
# )
# @expression(Expansion_Model, eNSECosts,
#      # Non-served energy costs
#     sum(sample_weight[t]*nse.NSE_Cost[s]*vNSE[t,s,z] for t in T, s in S, z in Z)
# )

For example, if a representative day represents a cluster with 8 total days, variable costs incurred in each hour in that representative period are multiplied by a `sample_weight` of 8, to represent repeated similar operational patterns in the clustered days. 

While time sampling methods like this can significantly reduce computational time, they also introduce abstraction errors that can bias model results. In this part of the assignment, we will explore these tradeoffs.

#### Question 1(a)

Copy the package commands and methods from `Lab1_code.jl` (except for `solve_cap_expansion_uc` which is not needed here).

In addition, copy the commands from `Lab1.jl` (load inputs, solve model, write results, etc.).

The model provided uses 10 representative days of data, and also records the solution time to a file in the output directory. 

However in the data folder (`Notebooks/complex_expansion_data`) there is also data for 4 weeks, 8 weeks, 16 weeks, and 52 weeks of representative periods.

**Run the model for four of these temporal resolutions.** Each time you run the model, (a) record the solution time, and (b) save your outputs to a different folder named `/10_days_Your_Name`, `/4_weeks_Your_Name`, `8_weeks_Your_Name`, and `16_weeks_Your_Name`.

Create and save a scatter plot that shows the solution time in seconds (y axis) and the number of hours included in the model for the 10 days (240 hours), 4 weeks (672 hours), 8 weeks (1344 hours), and 16 weeks (2688) iterations. What do you notice about the increase in solution time? How does the computational performance of the model appear to scale as the number of time steps in the model (hours) increases?

#### Question 1(b)

Now let's see how the results of the model compare. Compile tables that compare (i) total cost results, (ii) total final capacity (MW) results by resource, and (iii) the total generation (GWh) results for all four iterations of the model. 

What are the largest differences in results in each category? What do you think accounts for these differences? How would you measure or assess the accuracy of this model? How might this change, depending on the type of question are you are considering? 

#### Question 1(c)

The above experiment includes very little new wind or solar capacity additions. Let's try another case, which includes a **carbon tax of $50/tCO2**. 

The `generators` dataset contains information on the CO2 emissions per MWh (tCO2/MWh) in the column `CO2_Rate`.

Create a copy of the capacity expansion function below, name it `solve_cap_expansion_co2`, and modify the following:
- add a new parameter to the function `CO2_Price`
- add an additional expression `eCO2Costs` given by the following expression
- add this new cost to Total Costs

In [ ]:
#     @expression(Expansion_Model, eCO2Costs,
#          # Carbon tax costs: variable only
#         sum(sample_weight[t]*generators.CO2_Rate[g]*vGEN[t,g]*CO2_Price for t in T, g in G)
#     )

Now repeat the experiment in Question 1(a-b) above with the CO2 price, recording solution time and results for 10 days, 4 weeks, 8 weeks, and 16 weeks time series inputs. Save the results to `10_days_CO2_Your_Name`, etc.

Answer the following questions:

How have the capacity and energy results changed overall with addition of the carbon price to fuel costs (relative to the original cases? 

How does the variation in the cost, capacity, and energy ouputs change now as you consider different number/duration of sample periods?

What does the overall experiment in Question 1 tell you about the generalizability of time sampling methods?

#### Bonus

As a bonus question: you can run the full 52 week time series and compare to the sample time series for Question 1(b-c) and/or 1(d) above. This case may take up to an hour or more to solve depending on your CPU. How do the full year results differ from your reduced time sample cases? Anything surprising? What might running this tell you about extrapolating the performance of time sampling methods based on a reduced time series rather than a full year (e.g. comparing 10 days or 4 weeks to 16 weeks vs 52 weeks)? 

This is not a required question and is not worth additional credit, but do this if you're motivated/curious...

### Part 2: Clean electricity standard policy

For this question, in a new function `solve_cap_expansion_ces`, implement a clean electricity standard (CES) policy--a constraint that requires a certain share of generation from clean sources. Eligibility for a clean electricity standard is also recorded in the `generators.CES[g]` parameter and includes wind, solar, nuclear, generators with CCS, and hydro (not present).

Take care in your implementation to consider the role of time weights when using a reduced time series as we are here (not all hours are created equal!).

Run your capacity planning model with 8 weeks of data and under increasing stringency (e.g. at 0%, 20%, 40%, 60%, 80% and 100% clean energy requirement), and record and discuss results of each case. 

What do you notice about solution time and results as you change the stringency of the policy? How much does the policy increase costs at each level of stringency, relative to no policy? How much does it reduce CO$_2$ emissions? (You will need to add a calculation of CO$_2$ emissions to the model outputs recorded using `CO2_Rate`.)